In [ ]:
%pip install numpy
%pip install pymoo
%pip install deap
%pip install matplotlib
%pip install plotly
%pip install nbformat

In [ ]:
import numpy as np
from textwrap import wrap
from deap import benchmarks
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.core.problem import Problem

# _Genetic Algorithms_ <span style="margin-left: 10px; font-size: .6em;">an introduction</span>

For any question just drop me an email at luca.zanussi01@universitadipavia.it

### Define the problem

Here's a simple one, we want to minimize the following function in 6 variables (i.e., genes, in terms of the GA):

$$f(\mathbf{x}) := \left| \sum_{i=1}^6{x_i} - 42 \right| \qquad x_i \in [0, 20]$$

This translates in practice to the search for individuals in which genes, when summed, lead up to 42.
We also want to apply this constraint to the variables of the problem:

$$x_3^2 - x_1 + 2 = 0$$

In [ ]:
fn = lambda X: abs(sum(X) - 42)
co = lambda X: abs(X[2] ** 2 - X[0] + 2)

class MyProblem(Problem):
    def _evaluate(self, design, out, *args, **kwargs):
        out['F'] = np.array([ fn(X) for X in design ])
        out['G'] = np.array([ co(X) for X in design ])

problem = MyProblem(n_var=6, n_obj=1, n_ieq_constr=1, xl=[0]*6, xu=[20]*6)

### Run the Genetic Algorithm

Pymoo wants a _problem_, an _algorithm_ and a _stopping criteria_.
Here we wan to use the simple, basic GA with a population size of 100 and we stop it at its 100<sup>th</sup> generation.

In [ ]:
algorithm = GA(pop_size=100)
stop_criteria = ('n_gen', 100)

res = minimize(problem, algorithm, stop_criteria, seed=5, verbose=True)

> The [documentation of pymoo](https://pymoo.org/index.html) is quite good, you might want to check it out!
>
> Also, you can give a look at the code... see the [constructor of the GA class](https://github.com/anyoptimization/pymoo/blob/2ca840e30f49bb26c8ea796490eb7d345d557703/pymoo/algorithms/soo/nonconvex/ga.py#L58)? It takes a lot of different parameters among which the implementations of _crossover_, _mutation_ and _selection_, which can be changed.

#### Analyze the result

Here you can see the value of the genes, as well the minimized objective.

In [ ]:
print('Genes:     ', res.X)
print('Objective: ', res.F)

In fact, the sum of the genes is 42...

In [ ]:
sum(res.X)

...and the constraint is not violated.

In [ ]:
res.X[2] ** 2 - res.X[0] + 2

## _Bonus_: an alternative problem

There are lots of well-known benchmark optimization problems online,
see [Wikipedia](https://en.wikipedia.org/wiki/Test_functions_for_optimization) or the [Pymoo documentation](https://pymoo.org/problems/test_problems.html).

For example, let's see the **Ackley function**, it has a lot of global minima and it takes two variables in the range $[-32, 32]$.

In [ ]:
X = np.arange(-32, 32, .5)
Y = np.arange(-32, 32, .5)
X, Y = np.meshgrid(X, Y)
Z = np.empty(X.shape)

for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i, j] = benchmarks.ackley((X[i, j], Y[i, j]))[0]

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])
fig.update_layout(title='Ackley', autosize=False,
                  width=800, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

And define a problem for it, then run the GA.

In [ ]:
class AckleyProblem(Problem):
    def __init__(self, **kwargs):
        super().__init__(n_var=2, n_obj=1, xl=[-32]*2, xu=[32]*2, **kwargs)

    def _evaluate(self, design, out, *args, **kwargs):
        out['F'] = np.array([benchmarks.ackley(d) for d in design])

problem = AckleyProblem()
algorithm = GA(pop_size=100)
stop_criteria = ('n_gen', 100)

res = minimize(problem, algorithm, stop_criteria, seed=5, verbose=True)

The GA should have correctly found the global minimum at $\mathbf{x} = (0, 0)$.

In [ ]:
print('Genes:     ', res.X)
print('Objective: ', res.F)

## Multi-Objective optimization

Here we want to minimize the Kursawe functions, we use NSGA-II since this is a multi-objective problem.

In [ ]:
X = np.arange(-5, 5, .1)
Y = np.arange(-5, 5, .1)
X, Y = np.meshgrid(X, Y)

Z1 = np.empty(X.shape)
Z2 = np.empty(X.shape)

for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z1[i, j], Z2[i, j] = benchmarks.kursawe((X[i, j], Y[i, j]))

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    specs=[[{'is_3d': True}, {'is_3d': True}]],
                    subplot_titles=['Kursawe <i>f<sub>1</sub></i>', 'Kursawe <i>f<sub>2</sub></i>'])
fig.add_trace(go.Surface(x=X, y=Y, z=Z1, colorbar_x=-0.15), 1, 1)
fig.add_trace(go.Surface(x=X, y=Y, z=Z2), 1, 2)
fig.update_layout(height=600)
fig.show()

Let's define the problem for Pymoo and run NSGA-II

In [ ]:
class KursaweProblem(Problem):
    def __init__(self, **kwargs):
        super().__init__(n_var=2, n_obj=2, xl=[-5, -5], xu=[5, 5], **kwargs)

    def _evaluate(self, design, out, *args, **kwargs):
        out['F'] = np.array([benchmarks.kursawe(d) for d in design])

problem = KursaweProblem()
algorithm = NSGA2(pop_size=100)
stop_criteria = ('n_gen', 100)

res = minimize(problem, algorithm, stop_criteria, seed=5, verbose=True)

Differently from the single objective problem, here we have many solutions that do not dominate each other.

If you get 100 solutions, all individuals in the last generation got their place on the Pareto front; this might not always be the case, especially if you perform less iterations of the GA.

In [ ]:
len(res.X)

Let's pick a random solution and analyze it, we have 2 parameters (genes) and two objective values, one for each Kursawe function.

In [ ]:
print('Genes:      ', res.X[0])
print('Objectives: ', res.F[0])

As a last thing, we draw the Pareto front formed by all the non-dominated solutions to the problem.

In [ ]:
res_xy = res.F.T
res_ord = np.argsort(res_xy[0])  # This is to sort individuals by their f1 value and to be able to draw a nice line to connect them

fig = go.Figure(data=go.Scatter(x=res_xy[0,res_ord], y=res_xy[1,res_ord],
                                text=[f'<b>Genes: {"<br>".join(wrap(str(x)))}</b>' for x in res.X],
                                mode='lines+markers'))
fig.update_layout(width=800, height=600, title=f'NSGA-II solutions for {problem.name()}', xaxis_title='f1', yaxis_title='f2')
fig.show()

_That's All Folks!_